In [91]:
%load_ext autoreload
%autoreload 2

from arm_touchdown import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
def check_limits(joint_angles, min_constraints, max_constraints):
    if(np.alltrue(joint_angles>min_constraints) and 
       np.alltrue(joint_angles<max_constraints)):
        return True
    else:
        return False

In [93]:
# Run this cell everytime the arm is restarted
rospy.init_node('robot_control_node')

robot_controller = RobotController("Arm_RML", 
                                   ["J1_base", "J2_shoulder", "J3_elbow", 'J4_wrist'], 
                                   "/home/patrick/catkin_ws/src/hebi/arm_control/config/hrdf/anakin.hrdf")
eft = EndEffectorTrajectory(robot_controller)
print(robot_controller.get_joint_angles())
robot_controller.get_effort()

Gains path:  /home/patrick/catkin_ws/src/hebi/arm_control/config/gains/anakin_gains.xml
[ 4.73633242 -0.02016449 -3.01352787 -0.41283035]
[INFO] [1707342831.923378]: Effort: [ 0.5717211  -0.45296198 -0.05790221  0.26890376]


array([ 0.5717211 , -0.45296198, -0.05790221,  0.26890376], dtype=float32)

In [94]:
eft.positions = np.array([
   [ 4.74286461,  0.07531768, -2.95877361, -0.41710663], 
   [ 4.89363516,  0.46017654, -2.18277832, -0.37532043],
   [ 4.77134418,  2.00860817, -2.15998077, -0.29337311],
   [ 4.73502254,  2.63030655, -1.57555866, -0.37532043]
])

print(eft.positions)

[[ 4.74286461  0.07531768 -2.95877361 -0.41710663]
 [ 4.89363516  0.46017654 -2.18277832 -0.37532043]
 [ 4.77134418  2.00860817 -2.15998077 -0.29337311]
 [ 4.73502254  2.63030655 -1.57555866 -0.37532043]]


In [95]:
eft.get_trajectory(total_duration=8)
print(eft.trajectory.duration)


[ 4.73637581 -0.02020645 -3.01361752 -0.41279793]
8.0


In [96]:
cmd = hebi.GroupCommand(eft.num_joints)
        
period = 0.01
t = 0.0
duration = eft.trajectory.duration
while (t < duration):
        # self.robot_controller.group.get_next_feedback(reuse_fbk=self.group_fbk)
        eft.pos_cmd, eft.vel_cmd, eft.acc_cmd = eft.trajectory.get_state(t)
        
        effort = robot_controller.get_effort
        #if effort[2] < -4:
        #    cmd.mstop_strategy = 2
        
        cmd.position = eft.pos_cmd
        cmd.velocity = eft.vel_cmd
        eft.robot_controller.group.send_command(cmd)

        t = t + period
            

eft.execute_trajectory()


Period: 0.01
-----------
Sending command at time =  8.009999999999874
Current Trajectory State: (array([ 4.73502254,  2.63030655, -1.57555866, -0.37532043]), array([-9.02056208e-17,  1.11022302e-16,  3.33066907e-16, -1.04083409e-17]), array([-1.45716772e-16,  3.88578059e-16,  4.99600361e-16, -4.85722573e-17]))
Joint angles:  [ 4.75919437  2.77784378 -1.6389122  -0.41074371]
[INFO] [1707342855.262819]: Staying in the last position for 5 seconds...


In [7]:
# Working IK Values


------ POSITION ARRAY -------
[[ 4.73091793  4.89363516  4.77573206  4.74393144]
 [-0.02057076  0.46017654  1.16832948 -3.78152561]
 [-3.02600002 -2.18277832 -1.97059152 -1.77164562]
 [-2.28942299 -1.28403595 -0.78806982 -5.93687081]]
------------------------------

SyntaxError: invalid syntax (<ipython-input-7-c4d07caaa763>, line 3)

In [ ]:
[ 4.73091793, -0.02057076, -3.01241779, -0.37532043]


[ 4.74286461,  0.07531768, -2.95877361, -0.41710663], 
[ 4.89363516,  0.46017654, -2.18277832, -0.37532043],
[ 4.77134418,  2.00860817, -2.15998077, -0.29337311],
[ 4.73502254,  2.63030655, -1.57555866, -0.37532043]

In [5]:
# eft.target_positions = [
#            [0.03, 0.0, 0.4], # home 
#            [0.03, 0.2, 0.4], # extend
#            [0.03, 0.4, 0.0], # extend and move down
#             [0.0, 0.4, -0.1] 
#             only x,y,z coordinates
#        ]

# Original Constraints
# eft.min_positions = [4, -2*np.pi, -2*np.pi, -np.pi/2]
# eft.max_positions = [5, 2*np.pi, -1, np.pi/2]
            
# eft.min_positions = [4, -2*np.pi, -3.05, -np.pi]
# eft.max_positions = [5, 2*np.pi, -np.pi/4, np.pi]

# eft.get_trajectory(total_duration=40)

# print("------- POSITION ARRAY -------")
# print(eft.positions)
# print("------------------------------\n")


# for t in eft.times:
#    if not check_limits(eft.trajectory.get_state(t)[0], 
#                        np.array(eft.min_positions), 
#                        np.array(eft.max_positions)):
#        print(eft.trajectory.get_state(t)[0])
#        print("Time = ", t)
#        raise ("Joint Angles not within limits!")